# 單頁爬蟲

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

r    = requests.get("https://www.ptt.cc/bbs/koei/index.html") #將網頁資料GET下來
soup = bs(r.text) #將網頁資料以 html.parser
sel  = soup.select("div.title a") #取HTML標中的 <div class="title"></div> 中的<a>標籤存入sel
for s in sel: print("https://www.ptt.cc"+s["href"], s.text) #印出網址跟標題

# 跨頁爬蟲
## 最後一行縮排之後，即可多頁爬蟲

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
url = "https://www.ptt.cc/bbs/koei/index.html"
for i in range(3): #往上爬3頁
    r    = requests.get(url)
    soup = bs(r.text)
    sel  = soup.select("div.title a") #標題
    u    = soup.select("div.btn-group.btn-group-paging a") #a標籤
    print ("本頁的 URL 為 "+url)
    url  = "https://www.ptt.cc"+ u[1]["href"] #上一頁的網址
for s in sel: print("https://www.ptt.cc"+s["href"], s.text) #印出網址跟標題

# researchgate 爬蟲 & 寫入 excel

In [26]:
%%time
import requests,xlsxwriter
from bs4 import BeautifulSoup as bs

# Login researchgate
url = 'https://www.researchgate.net/application.Login.html'
session = requests.Session()
session.post(url,data = {
            'request_token':bs(session.get(url).text).form.find("input",{"name":"request_token"}).attrs["value"],
            'login'   :'reganlu007@ntu.edu.tw', # Username
            'password':'fbqh1520e',             # Password
            'setLoginCookie':'yes'})            # Save log cookie

name = 'Francis_Narin'
url  = 'https://www.researchgate.net/profile/'+ name # Search 'Francis_Narin'
soup = bs(session.get(url).text)

# Create/Open Research_Gate_Result.xlsx
workbook  = xlsxwriter.Workbook('Research_Gate_Result.xlsx',{'strings_to_numbers': True})

# Overview: Introduction, Stats overview, Affiliation, Network, Top co-authors, Research overview
Overview = soup.findAll(True,{'nova-e-text nova-e-text--size-l nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-grey-600'})

# Sheet 1: Info
worksheet = workbook.add_worksheet('Info')
val = ['Name','Item','Key','Value']
for i in range(len(val)): worksheet.write(0,i,val[i],workbook.add_format({'bold':True,'center_across':True}))
val = [ name ,Overview[0].text,'RG Score',soup.findAll(True,{'nova-e-list__item'})[0].text]
for i in range(len(val)): worksheet.write(1,i,val[i])
curr_len = 2

## Stats overview
key = soup.findAll(True,{'nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-xxs nova-e-text--color-inherit'})
val = soup.findAll(True,{'nova-e-text nova-e-text--size-xl nova-e-text--family-sans-serif nova-e-text--spacing-xxs nova-e-text--color-inherit'})
for i in range(curr_len,curr_len+len(val)):
    worksheet.write(i,0,name)
    worksheet.write(i,1,Overview[1].text)
    worksheet.write(i,2,key[i-curr_len].text)
    worksheet.write(i,3,val[i-curr_len].text.replace(',',''))
curr_len += len(val)

## Affiliation
key = soup.findAll(True,{'nova-e-text nova-e-text--size-l nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-grey-600'})
val = soup.findAll(True,{'nova-e-text nova-e-text--size-l nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit nova-v-institution-item__title'})
for i in range(curr_len,curr_len+len(val)):
    worksheet.write(i,0,name)
    worksheet.write(i,1,Overview[2].text)
    worksheet.write(i,2,key[i-curr_len].text)
    worksheet.write(i,3,val[i-curr_len].text)
curr_len += len(val)

## Research overview
key = soup.findAll(True,{'nova-e-text nova-e-text--size-m nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit'})
val = soup.findAll(True,{'nova-e-text nova-e-text--size-xl nova-e-text--family-sans-serif nova-e-text--spacing-none nova-e-text--color-inherit'})
for i in range(curr_len,curr_len+len(val)):
    worksheet.write(i,0,name)
    worksheet.write(i,1,Overview[5].text)
    worksheet.write(i,2,key[i-curr_len+3].text)
    worksheet.write(i,3,val[i-curr_len].text)
curr_len += len(val)

# Sheet 2: Skills & Expertise
worksheet = workbook.add_worksheet('Skills & Expertise')
val = ['Name','SEQ','Item']
for i in range(len(val)): worksheet.write(0,i,val[i],workbook.add_format({'bold':True,'center_across':True}))
val = bs(session.get(url +'/info').text).findAll(True,{'nova-e-badge nova-e-badge--color-grey nova-e-badge--display-block nova-e-badge--luminosity-medium nova-e-badge--size-l nova-e-badge--theme-ghost nova-e-badge--radius-full'})
for i in range(1,1 +len(val)):
    worksheet.write(i,0,name)
    worksheet.write(i,1,i)
    worksheet.write(i,2,val[i-1].text)

# Sheet 3: Co Author
worksheet = workbook.add_worksheet('Co Author')
val = ['Name','Co Author','RG Score']
for i in range(len(val)):
    worksheet.write(0,i,val[i],workbook.add_format({'bold':True,'center_across':True}))
val = soup.findAll(True,{'nova-e-link nova-e-link--color-inherit nova-e-link--theme-bare'})
for i in range(1,1 + 5):
    worksheet.write(i,0,name)
    worksheet.write(i,1,val[i*2+5].text)
    worksheet.write(i,2,val[i*2+6].text)

workbook.close()

Wall time: 6.99 s
